In [1]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"thnhnguyenphuc","key":"c4567d6a802976fa686fb1f293c5ce5f"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

# https://technowhisp.com/kaggle-api-python-documentation/
# Authenticate to Kaggle
api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('duyminhnguyentran/csc15105', unzip=True)

In [3]:
file_path_train = "Project2_Data/train.json"
file_path_test = "Project2_Data/test.json"

In [4]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 52.0

In [5]:
import json
with open(file_path_train, "r") as read_file:
    train = json.load(read_file)

In [6]:
with open(file_path_test, "r") as read_file:
    test = json.load(read_file)

In [7]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [8]:
model_type="squeezebert"
model_name= "squeezebert/squeezebert-uncased"


In [9]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [10]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "train_batch_size": 64,
    "eval_batch_size": 32,
}


In [11]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
model.train_model(train, eval_data=test)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
convert squad examples to features:   0%|          | 0/50046 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Could not find answer: '' vs. 'Crowborough,'
Could not find answer: '' vs. 'Đông Sussex'
Could not find answer: '' vs. 'tăng'
Could not find answer: '' vs. 'trong và ngoài nước'
convert squad examples to features:   0%|          | 1/50046 [02:47<2329:32:19, 167.58s/it]Could not find answer: '' vs. 'chất ức chế ACE'
Could not find answer: '' vs. 'làm giảm nguy cơ bệnh thận mới bắt đầu [RR 0'
Could not find answer: '' vs. 'Nhật Bản'
add example index and un

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Evaluation:   0%|          | 0/524 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/820 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/524 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/524 [00:00<?, ?it/s]

(4100,
 {'global_step': [820, 1000, 1640, 2000, 2460, 3000, 3280, 4000, 4100],
  'correct': [604, 643, 957, 931, 1077, 1095, 1196, 1263, 1247],
  'similar': [12507, 12660, 11504, 12116, 10878, 11382, 10818, 10822, 10956],
  'incorrect': [2883, 2691, 3533, 2947, 4039, 3517, 3980, 3909, 3791],
  'train_loss': [4.220703125,
   2.597614288330078,
   3.415663003921509,
   2.340546131134033,
   2.7654213905334473,
   3.0172901153564453,
   3.1270885467529297,
   2.821845054626465,
   2.1220498085021973],
  'eval_loss': [-1.4495234598640268,
   -1.2098155713263359,
   -1.6892909858062977,
   -1.8377862595419847,
   -1.6464620109732824,
   -1.8789863877624047,
   -1.7259046248807253,
   -1.7887214485925573,
   -1.7786557729007633]})

In [13]:
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/524 [00:00<?, ?it/s]

In [14]:
result

{'correct': 1247,
 'similar': 10956,
 'incorrect': 3791,
 'eval_loss': -1.7786557729007633}

In [15]:
texts

{'correct_text': {'5733bf84d058e614000b61bd': 'ba',
  '5733afd3d058e614000b6046': 'USA Today $ $$ xếp Notre Dame 10 tổng thể cho các trường đại học Mỹ dựa trên dữ liệu từ Cao đẳng Thực tế. Forbes. Best Colleges của com của Mỹ đứng thứ Notre Dame thứ 13 trong số các trường đại học tại Hoa Kỳ vào năm 2015,',
  '5733b2fe4776f41900661090': '1928',
  '5733b3d64776f419006610a5': '44',
  '5733b5df4776f41900661109': '12.179',
  '5733b5df4776f41900661106': '21-24%',
  '5733b699d058e614000b611a': '15',
  '573398ebd058e614000b5e67': '1950',
  '57339a5bd058e614000b5e91': '29',
  '57339b36d058e614000b5ea5': '',
  '573382a14776f41900660c2e': '',
  '5733849bd058e614000b5c59': 'ngày 26 tháng 11 năm 1842,',
  '573385394776f41900660c7f': '1849',
  '5733c0064776f4190066119a': 'WSND-FM',
  '5733c4494776f419006611da': '',
  '5733caf74776f4190066124d': '12',
  '56cef8faaab44d1400b88d6a': '04 tháng 9 năm 1981)',
  '56d43da72ccc5a1400d830be': "B'Day",
  '56d43da72ccc5a1400d830bf': 'Dreamgirls',
  '56bf76ef3ae

In [16]:
# Make predictions with the model
to_predict = [
    {
        "context": "Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông, hai nguồn năng lượng tái tạo hiệu quả nhất. Kể từ thời điểm chuyển giao thế kỷ 21, đã có một xu hướng phát triển của một ngành công nghiệp tài nguyên tái tạo và giảm cả tiêu thụ và sử dụng tài nguyên nhiên liệu hóa thạch. Năm 2006, nhà máy điện năng lượng mặt trời lớn nhất thế giới tại thời điểm đó, nhà máy điện Moura quang điện, bắt đầu hoạt động gần Moura, ở phía nam, trong khi trang trại năng lượng sóng thương mại đầu tiên trên thế giới, Aguçadoura sóng trại , mở trong khu vực Norte  (2008). Đến cuối năm 2006, 66% sản lượng điện của nước này là từ nhà máy than và điện nhiên liệu, trong khi 29% được lấy từ đập thủy điện, và 6% bằng năng lượng gió. .",
        "qas": [
            {
                "question": "hai nguồn đáng kể nhất của năng lượng tái tạo ở Bồ Đào Nha là gì?",
                "id": "0",
            }
        ],
    }
]

In [17]:
answers, probabilities = model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3109.19it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông, hai nguồn năng lượng tái tạo hiệu quả nhất.', 'Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông,', 'quả nhất.']}]
